# Data prepare and FC_DNN

In [22]:
# Import data_prepare modul that contains functions that we created.
from data_prepare import *
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

# Prepare data for learning

We prepare the data for learning neural networks. We use a function that execute the results of the last milestone get a train-ready dataset.

In [49]:
def generate_datasets(pt, feat_path=r'./features'):
    
    r"""
    Generate train and test datasets from the one sample of the raw data.
    param pt: name of the sample
    param feat_path: path of the raw data file
  """
    
    spectrogram = np.load(os.path.join(feat_path,f'{pt}_spec.npy'))
    data = np.load(os.path.join(feat_path,f'{pt}_feat.npy'))
    
    # Reduce Dimensions
    pca = PCA()
    pca.fit(data)
    data = np.dot(data, pca.components_[:50,:].T)

    # Generate train, validation and test datasets
    X_train, X_test, y_train, y_test = train_test_split(data, spectrogram, test_size=0.1)
    
    # Standardize data
    mu = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)
    X_train = (X_train-mu)/std
    X_test = (X_test-mu)/std

    return (X_train, X_test, y_train, y_test)

In [57]:
def generate_merged_dataset():
    pts = ['sub-%02d'%i for i in range(1,11)]
    (X_train, X_test, y_train, y_test) = generate_datasets(pts[0])
    for pt in ['sub-%02d'%i for i in range(1,11)][1:]:
        (XN_train, XN_test, yN_train, yN_test) = generate_datasets(pt)
        X_tarin = np.concatenate((X_train, XN_train))
        X_test = np.concatenate((X_test, XN_test))
        y_tarin = np.concatenate((y_train, yN_train))
        y_test = np.concatenate((y_test, yN_test))
    return (X_train, X_test, y_train, y_test)

In [58]:
(X_train, X_test, y_train, y_test) = generate_merged_dataset()

# Create FC-DNN network

In [59]:
def create_fndnn(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(units=60, input_dim=input_dim, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(units=30, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(units=output_dim, activation='linear'))

    optimizer=Adam(lr=0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    return model

In [60]:
epochs = 1000
batch_size = 128

early_stopping=EarlyStopping(patience=25, verbose=1)
checkpointer=ModelCheckpoint(filepath='fcdnn.hdf5', save_best_only=True, verbose=1)

model = create_fndnn(input_dim=X_train.shape[1], output_dim=y_train.shape[1])

history = model.fit(X_train,y_train, validation_split=0.1, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[checkpointer, early_stopping])

Epoch 1/1000


C:\Users\Borkó Károly\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Epoch 1: val_loss improved from inf to 4.04021, saving model to fcdnn.hdf5
190/190 - 1s - loss: 13.1319 - mae: 2.8318 - val_loss: 4.0402 - val_mae: 1.2670 - 1s/epoch - 6ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 4.04021 to 3.38585, saving model to fcdnn.hdf5
190/190 - 0s - loss: 5.2244 - mae: 1.6780 - val_loss: 3.3858 - val_mae: 1.3349 - 473ms/epoch - 2ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 3.38585 to 3.33799, saving model to fcdnn.hdf5
190/190 - 0s - loss: 4.8392 - mae: 1.6270 - val_loss: 3.3380 - val_mae: 1.3464 - 479ms/epoch - 3ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 3.33799 to 3.30572, saving model to fcdnn.hdf5
190/190 - 1s - loss: 4.6535 - mae: 1.5853 - val_loss: 3.3057 - val_mae: 1.3198 - 577ms/epoch - 3ms/step
Epoch 5/1000

Epoch 5: val_loss did not improve from 3.30572
190/190 - 1s - loss: 4.5151 - mae: 1.5562 - val_loss: 3.3065 - val_mae: 1.2841 - 568ms/epoch - 3ms/step
Epoch 6/1000

Epoch 6: val_loss improved from 3.30572 to 3.28204, 

190/190 - 1s - loss: 3.5740 - mae: 1.3816 - val_loss: 3.2196 - val_mae: 1.3440 - 598ms/epoch - 3ms/step
Epoch 45/1000

Epoch 45: val_loss improved from 3.21952 to 3.21678, saving model to fcdnn.hdf5
190/190 - 1s - loss: 3.6054 - mae: 1.3888 - val_loss: 3.2168 - val_mae: 1.3186 - 614ms/epoch - 3ms/step
Epoch 46/1000

Epoch 46: val_loss improved from 3.21678 to 3.21624, saving model to fcdnn.hdf5
190/190 - 1s - loss: 3.5683 - mae: 1.3824 - val_loss: 3.2162 - val_mae: 1.3347 - 566ms/epoch - 3ms/step
Epoch 47/1000

Epoch 47: val_loss did not improve from 3.21624
190/190 - 1s - loss: 3.5586 - mae: 1.3799 - val_loss: 3.2247 - val_mae: 1.3607 - 550ms/epoch - 3ms/step
Epoch 48/1000

Epoch 48: val_loss improved from 3.21624 to 3.21180, saving model to fcdnn.hdf5
190/190 - 1s - loss: 3.5592 - mae: 1.3817 - val_loss: 3.2118 - val_mae: 1.3349 - 584ms/epoch - 3ms/step
Epoch 49/1000

Epoch 49: val_loss improved from 3.21180 to 3.20857, saving model to fcdnn.hdf5
190/190 - 1s - loss: 3.5388 - mae: 1.

Epoch 87/1000

Epoch 87: val_loss improved from 3.10332 to 3.09867, saving model to fcdnn.hdf5
190/190 - 1s - loss: 3.3566 - mae: 1.3522 - val_loss: 3.0987 - val_mae: 1.3110 - 504ms/epoch - 3ms/step
Epoch 88/1000

Epoch 88: val_loss did not improve from 3.09867
190/190 - 0s - loss: 3.3677 - mae: 1.3547 - val_loss: 3.0990 - val_mae: 1.2947 - 479ms/epoch - 3ms/step
Epoch 89/1000

Epoch 89: val_loss improved from 3.09867 to 3.09608, saving model to fcdnn.hdf5
190/190 - 0s - loss: 3.3706 - mae: 1.3539 - val_loss: 3.0961 - val_mae: 1.3072 - 489ms/epoch - 3ms/step
Epoch 90/1000

Epoch 90: val_loss improved from 3.09608 to 3.09369, saving model to fcdnn.hdf5
190/190 - 1s - loss: 3.3686 - mae: 1.3541 - val_loss: 3.0937 - val_mae: 1.3068 - 535ms/epoch - 3ms/step
Epoch 91/1000

Epoch 91: val_loss improved from 3.09369 to 3.08981, saving model to fcdnn.hdf5
190/190 - 1s - loss: 3.3512 - mae: 1.3523 - val_loss: 3.0898 - val_mae: 1.2962 - 568ms/epoch - 3ms/step
Epoch 92/1000

Epoch 92: val_loss imp

...

190/190 - 0s - loss: 2.9904 - mae: 1.2468 - val_loss: 2.6856 - val_mae: 1.1734 - 251ms/epoch - 1ms/step
Epoch 407/1000

Epoch 407: val_loss did not improve from 2.67450
190/190 - 0s - loss: 2.9889 - mae: 1.2432 - val_loss: 2.6779 - val_mae: 1.1738 - 240ms/epoch - 1ms/step
Epoch 408/1000

Epoch 408: val_loss improved from 2.67450 to 2.67145, saving model to fcdnn.hdf5
190/190 - 0s - loss: 2.9737 - mae: 1.2429 - val_loss: 2.6714 - val_mae: 1.1747 - 267ms/epoch - 1ms/step
Epoch 409/1000

Epoch 409: val_loss did not improve from 2.67145
190/190 - 0s - loss: 2.9916 - mae: 1.2452 - val_loss: 2.6870 - val_mae: 1.1688 - 241ms/epoch - 1ms/step
Epoch 410/1000

Epoch 410: val_loss did not improve from 2.67145
190/190 - 0s - loss: 2.9917 - mae: 1.2431 - val_loss: 2.6760 - val_mae: 1.1671 - 235ms/epoch - 1ms/step
Epoch 411/1000

Epoch 411: val_loss did not improve from 2.67145
190/190 - 0s - loss: 2.9934 - mae: 1.2461 - val_loss: 2.6899 - val_mae: 1.1710 - 241ms/epoch - 1ms/step
Epoch 412/1000

Epo

Epoch 454/1000

Epoch 454: val_loss did not improve from 2.64632
190/190 - 0s - loss: 2.9780 - mae: 1.2395 - val_loss: 2.6671 - val_mae: 1.1592 - 251ms/epoch - 1ms/step
Epoch 455/1000

Epoch 455: val_loss did not improve from 2.64632
190/190 - 0s - loss: 2.9654 - mae: 1.2401 - val_loss: 2.6566 - val_mae: 1.1622 - 241ms/epoch - 1ms/step
Epoch 456/1000

Epoch 456: val_loss did not improve from 2.64632
190/190 - 0s - loss: 2.9836 - mae: 1.2414 - val_loss: 2.6628 - val_mae: 1.1717 - 235ms/epoch - 1ms/step
Epoch 457/1000

Epoch 457: val_loss did not improve from 2.64632
190/190 - 0s - loss: 2.9828 - mae: 1.2421 - val_loss: 2.6585 - val_mae: 1.1671 - 241ms/epoch - 1ms/step
Epoch 458/1000

Epoch 458: val_loss improved from 2.64632 to 2.63892, saving model to fcdnn.hdf5
190/190 - 0s - loss: 2.9526 - mae: 1.2368 - val_loss: 2.6389 - val_mae: 1.1731 - 267ms/epoch - 1ms/step
Epoch 459/1000

Epoch 459: val_loss did not improve from 2.63892
190/190 - 0s - loss: 2.9467 - mae: 1.2357 - val_loss: 2.65

190/190 - 0s - loss: 2.9324 - mae: 1.2318 - val_loss: 2.6298 - val_mae: 1.1565 - 235ms/epoch - 1ms/step
Epoch 502/1000

Epoch 502: val_loss did not improve from 2.62508
190/190 - 0s - loss: 2.9483 - mae: 1.2334 - val_loss: 2.6326 - val_mae: 1.1523 - 241ms/epoch - 1ms/step
Epoch 503/1000

Epoch 503: val_loss improved from 2.62508 to 2.62129, saving model to fcdnn.hdf5
190/190 - 0s - loss: 2.9263 - mae: 1.2243 - val_loss: 2.6213 - val_mae: 1.1607 - 251ms/epoch - 1ms/step
Epoch 504/1000

Epoch 504: val_loss improved from 2.62129 to 2.62049, saving model to fcdnn.hdf5
190/190 - 0s - loss: 2.9430 - mae: 1.2327 - val_loss: 2.6205 - val_mae: 1.1601 - 272ms/epoch - 1ms/step
Epoch 505/1000

Epoch 505: val_loss improved from 2.62049 to 2.62033, saving model to fcdnn.hdf5
190/190 - 0s - loss: 2.9483 - mae: 1.2337 - val_loss: 2.6203 - val_mae: 1.1558 - 266ms/epoch - 1ms/step
Epoch 506/1000

Epoch 506: val_loss did not improve from 2.62033
190/190 - 0s - loss: 2.9528 - mae: 1.2337 - val_loss: 2.650

Epoch 549/1000

Epoch 549: val_loss did not improve from 2.60876
190/190 - 0s - loss: 2.9486 - mae: 1.2315 - val_loss: 2.6228 - val_mae: 1.1520 - 236ms/epoch - 1ms/step
Epoch 550/1000

Epoch 550: val_loss did not improve from 2.60876
190/190 - 0s - loss: 2.9239 - mae: 1.2269 - val_loss: 2.6173 - val_mae: 1.1522 - 240ms/epoch - 1ms/step
Epoch 551/1000

Epoch 551: val_loss improved from 2.60876 to 2.60679, saving model to fcdnn.hdf5
190/190 - 0s - loss: 2.9366 - mae: 1.2279 - val_loss: 2.6068 - val_mae: 1.1544 - 251ms/epoch - 1ms/step
Epoch 552/1000

Epoch 552: val_loss did not improve from 2.60679
190/190 - 0s - loss: 2.9303 - mae: 1.2279 - val_loss: 2.6182 - val_mae: 1.1561 - 240ms/epoch - 1ms/step
Epoch 553/1000

Epoch 553: val_loss did not improve from 2.60679
190/190 - 0s - loss: 2.9308 - mae: 1.2317 - val_loss: 2.6180 - val_mae: 1.1552 - 235ms/epoch - 1ms/step
Epoch 554/1000

Epoch 554: val_loss improved from 2.60679 to 2.60435, saving model to fcdnn.hdf5
190/190 - 0s - loss: 2.921

Epoch 596/1000

Epoch 596: val_loss did not improve from 2.58015
190/190 - 0s - loss: 2.8964 - mae: 1.2187 - val_loss: 2.6074 - val_mae: 1.1417 - 252ms/epoch - 1ms/step
Epoch 597/1000

Epoch 597: val_loss did not improve from 2.58015
190/190 - 0s - loss: 2.9329 - mae: 1.2270 - val_loss: 2.5933 - val_mae: 1.1476 - 241ms/epoch - 1ms/step
Epoch 598/1000

Epoch 598: val_loss did not improve from 2.58015
190/190 - 0s - loss: 2.9067 - mae: 1.2217 - val_loss: 2.6036 - val_mae: 1.1430 - 251ms/epoch - 1ms/step
Epoch 599/1000

Epoch 599: val_loss did not improve from 2.58015
190/190 - 0s - loss: 2.9054 - mae: 1.2202 - val_loss: 2.5859 - val_mae: 1.1472 - 241ms/epoch - 1ms/step
Epoch 600/1000

Epoch 600: val_loss did not improve from 2.58015
190/190 - 0s - loss: 2.9307 - mae: 1.2240 - val_loss: 2.5969 - val_mae: 1.1497 - 236ms/epoch - 1ms/step
Epoch 601/1000

Epoch 601: val_loss did not improve from 2.58015
190/190 - 0s - loss: 2.9229 - mae: 1.2258 - val_loss: 2.6078 - val_mae: 1.1412 - 238ms/epo

In [61]:
y_pred = model.predict(X_test)

933/933 [==============================] - 1s 505us/step


In [62]:
loss = mean_absolute_error(y_test, y_pred)
print(f'Mean absolute error of test set: {loss}')

Mean absolute error of test set: 1.667150793969248
